In [1]:
import breakwater as bw
import pandas as pd
import os
from pathlib import Path

In [7]:
project_data = pd.read_excel(Path("./Input data/") / "test_data_phase_II.xlsx", 
                             sheet_name='Input_Project specific')
requirements_data = pd.read_excel(Path("./Input data/") / "test_data_phase_II.xlsx", 
                                  sheet_name='Input_requirements')
wave_data = pd.read_excel(Path("./Input data/") / "test_data_phase_II.xlsx", 
                          sheet_name='input_hydrotechnical',
                         skiprows = 1)
cross_section_data = pd.read_excel(Path("./Input data/") / "test_data_phase_II.xlsx", 
                                   sheet_name='Input_Cross section',
                                  skiprows = 1)

In [3]:
project_data

,Parameter,Symbol,Unit,Explanation,Value
0,Water density,rho_w,kg/m3,NaN,1010
1,Rock density,rho_r,kg/m3,NaN,2600
2,Concrete density,rho_c,kg/m3,NaN,2350
3,Gravity,g,m/s^2,NaN,9.81
4,Exploitation limit state,ELS,-,NaN,ELS
5,Servicability limit state,SLS,-,NaN,SLS
6,Ultimate limit state,ULS,-,NaN,ULS
7,Plunging coefficient,cpl,-,NaN,6.2
8,Surging coefficient,cs,-,NaN,1
9,Number of waves,N,-,NaN,3000


In [4]:
requirements_data

,Parameter,Symbol,Unit,Explanation,ELS,SLS,ULS
0,Overtopping limit,q_max,l/m/s,NaN,0.1,5.0,200.0
1,Armour damage limit,Sd_max,-,NaN,NaN,NaN,NaN
2,Toe damage limit,Nod_max_toe,-,NaN,NaN,NaN,NaN
3,Concrete armour damage limit,Nod_max_armour,-,NaN,NaN,NaN,NaN


In [5]:
wave_data

,Structure,Chainage,Offshore bin,Limit state,wl,Hm0,dir_wave,Tp,"Tm-1,0","Tm0,2"
0,ECn2,370N,0,ELS,0.38,1.060,48.1,5.58,3.95,2.78
1,ECn2,370N,30,ELS,0.47,2.060,69.1,8.11,6.66,4.85
2,ECn2,370N,60,ELS,0.56,2.800,78.9,9.01,8.05,6.60
3,ECn2,370N,90,ELS,0.50,2.570,86.5,9.00,7.85,6.30
4,ECn2,370N,120,ELS,0.39,1.650,98.0,7.07,6.02,4.77
...,...,...,...,...,...,...,...,...,...,...
163,ECn2,120N,90,ULS,0.85,1.908,72.7,12.03,11.73,10.95
164,ECn2,120N,120,ULS,0.63,1.584,74.7,9.89,9.49,8.72
165,ECn2,120N,150,ULS,0.65,1.464,78.1,9.62,8.98,7.99
166,ECn2,120N,180,ULS,0.55,0.768,79.8,8.21,7.37,4.95


In [8]:
cross_section_data


,Structure,Chainage,dir_structure,z_bed,public,P,tan_a,Gc,slope_foreshore,gamma_f_c,cot_a_rear,B,roundhead
0,ECn2,370N,85,-6.6,0,0.4,0.75,1.5,0.01,0.55,1.5,6,1
1,ECn2,320N,85,-5.9,0,0.4,0.67,1.5,0.01,0.55,1.5,6,0
2,ECn2,270N,85,-4.0,0,0.4,0.67,1.5,0.01,0.55,1.5,6,0
3,ECn2,220N,85,-3.5,0,0.4,0.67,1.5,0.01,0.55,1.5,6,0
4,ECn2,190N,85,-3.0,0,0.4,0.67,1.5,0.01,0.55,1.5,6,0
5,ECn2,170N,85,-2.0,1,0.4,0.67,1.5,0.01,0.55,1.5,6,0
6,ECn2,120N,85,-1.2,1,0.4,0.67,1.5,0.01,0.55,1.5,6,0


In [ ]:
# Calculate crest height for overtopping

#Calculate xi


# Calculate gamma_f
